In [5]:
import requests
import pandas as pd
import numpy as np
import datetime

def etherscan_request(action, 
                      address='0x7bea39867e4169dbe237d55c8242a8f2fcdcc387', 
                      api_key='Y8YP3J3BTG1DRZZPQ3F2I41E88I9GSUXNU', 
                      startblock=0, 
                      endblock=99999999, 
                      sort='desc'):
    
    base_url = 'https://api.etherscan.io/api'
    params = {
        'module': 'account',
        'action': action,
        'address': address,
        'startblock': startblock,
        'endblock': endblock,
        'sort': sort,
        'apikey': api_key
    }
    
    response = requests.get(base_url, params=params)
    
    if response.status_code != 200:
        raise Exception(f"API request failed with status code {response.status_code}")
    
    data = response.json()
    
    if data['status'] != '1':
        raise Exception(f"API returned an error: {data['result']}")
    
    df = pd.DataFrame(data['result'])

    expected_columns = ['hash', 'blockNumber', 'timeStamp', 'from', 'to', 'gas', 'gasPrice', 'gasUsed', 'cumulativeGasUsed', 'confirmations', 'tokenSymbol', 'value', 'tokenName']
    
    for col in expected_columns:
        if col not in df.columns:
            raise Exception(f"Expected column '{col}' is missing from the response")
    
    df['value'] = pd.to_numeric(df['value'], errors='coerce')
    
    # Set Transaction Value in Appropriate Format
    df['og_value'] = df['value'].copy()
    df['value'] = np.where(df['tokenDecimal']=='6', df['value']/100000, df['value']/1000000000000000000)

    # Sort by timestamp in descending order and select the most recent 10,000 trades
    df['timeStamp'] = pd.to_numeric(df['timeStamp'])
    df = df.sort_values(by='timeStamp', ascending=False).head(10000)
    
    consolidated_data = {}

    for index, row in df.iterrows():
        tx_hash = row['hash']
        
        if tx_hash not in consolidated_data:
            consolidated_data[tx_hash] = {
                'blockNumber': row['blockNumber'],
                'timeStamp': row['timeStamp'],
                'hash': tx_hash,
                'from': row['from'],
                'to': row['to'],
                'WETH_value': 0,
                'USDC_value': 0,
                'tokenName_WETH': '',
                'tokenName_USDC': '',
                'gas': row['gas'],
                'gasPrice': row['gasPrice'],
                'gasUsed': row['gasUsed'],
                'cumulativeGasUsed': row['cumulativeGasUsed'],
                'confirmations': row['confirmations']
            }
        
        if row['tokenSymbol'] == 'WETH':
            consolidated_data[tx_hash]['WETH_value'] = row['value']
            consolidated_data[tx_hash]['tokenName_WETH'] = row['tokenName']
        elif row['tokenSymbol'] == 'USDC':
            consolidated_data[tx_hash]['USDC_value'] = row['value']
            consolidated_data[tx_hash]['tokenName_USDC'] = row['tokenName']

    return pd.DataFrame.from_dict(consolidated_data, orient='index'), df

consolidated, transactions = etherscan_request('tokentx')
consolidated['time'] = consolidated['timeStamp'].apply(lambda x:datetime.datetime.fromtimestamp(x))
consolidated


,blockNumber,timeStamp,hash,from,to,WETH_value,USDC_value,tokenName_WETH,tokenName_USDC,gas,gasPrice,gasUsed,cumulativeGasUsed,confirmations,time
0xe1f15e9e24d6b3bb34ab9ad3c1195d1c88a2aa19b5c1fda68ba3a52416762909,20406852,1722191423,0xe1f15e9e24d6b3bb34ab9ad3c1195d1c88a2aa19b5c1...,0x51c72848c68a965f66fa7a88855f9f7784502a7f,0x7bea39867e4169dbe237d55c8242a8f2fcdcc387,0.619691,20334.06966,Wrapped Ether,USDC,164122,2013236823,131358,2988581,399,2024-07-28 11:30:23
0x3565bb4754cfc88d04a5c2b37875c02313853ad1fc28fe03ef004e1dd3b4077d,20406385,1722185807,0x3565bb4754cfc88d04a5c2b37875c02313853ad1fc28...,0x7bea39867e4169dbe237d55c8242a8f2fcdcc387,0xc36442b4a4522e871399cd717abdd847ab11fe88,0.055771,3507.16116,Wrapped Ether,USDC,474582,2006887568,313889,6925378,866,2024-07-28 09:56:47
0x69f9329bba83395d3fa2169faac7baad2e041c2b59a392a3f8396043737d67a6,20405459,1722174671,0x69f9329bba83395d3fa2169faac7baad2e041c2b59a3...,0xea749fd6ba492dbc14c24fe8a3d08769229b896c,0x7bea39867e4169dbe237d55c8242a8f2fcdcc387,0.299858,9826.38861,Wrapped Ether,USDC,346555,2914933007,277528,8032309,1792,2024-07-28 06:51:11
0x4fd5395803d038bf497583db9de3d6408a4e1c15b65644e08b11ebb3718b322a,20405445,1722174503,0x4fd5395803d038bf497583db9de3d6408a4e1c15b656...,0x4313c378cc91ea583c91387b9216e2c03096b27f,0x7bea39867e4169dbe237d55c8242a8f2fcdcc387,0.004950,162.15082,Wrapped Ether,USDC,392369,2162216925,339663,11555384,1806,2024-07-28 06:48:23
0x6d44dc3d2d0526e08fcbc47c0e237cd7d49cbb7bebba8c3e163ca36c66c22a91,20405224,1722171851,0x6d44dc3d2d0526e08fcbc47c0e237cd7d49cbb7bebba...,0xe37e799d5077682fa0a244d46e5649f71457bd09,0x7bea39867e4169dbe237d55c8242a8f2fcdcc387,0.235443,7709.50378,Wrapped Ether,USDC,312933,1312021778,259403,22300151,2027,2024-07-28 06:04:11
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
0x7f8b56f0648c9db1205147d2eba7df0329922a61f62043a99eae5c7aa575596b,19272643,1708477259,0x7f8b56f0648c9db1205147d2eba7df0329922a61f620...,0xdef1c0ded9bec7f1a1670819833240f027b25eff,0x7bea39867e4169dbe237d55c8242a8f2fcdcc387,0.000147,4.35792,Wrapped Ether,USDC,310919,45000000000,194141,4150919,1134608,2024-02-20 17:00:59
0x7784f58553d0550bf867000b3b4880f48b4173a413436afea157fe0955a26de7,19272498,1708475507,0x7784f58553d0550bf867000b3b4880f48b4173a41343...,0x636aaa78c95fc07525c9614028a2bf51737bc59b,0x7bea39867e4169dbe237d55c8242a8f2fcdcc387,0.001650,50.00000,Wrapped Ether,USDC,261667,35468918246,191400,2708272,1134753,2024-02-20 16:31:47
0xf5bd4ddc5d081a9c63255bf9dc81cc423affb10e3867a523e73c963e8fe23b41,19272125,1708470995,0xf5bd4ddc5d081a9c63255bf9dc81cc423affb10e3867...,0x6f1cdbbb4d53d226cf4b917bf768b94acbab6168,0x7bea39867e4169dbe237d55c8242a8f2fcdcc387,4.970655,150366.04773,Wrapped Ether,USDC,182780,55293498826,113832,17373406,1135126,2024-02-20 15:16:35
0x7c261729906516b7667cf37eb99fb0a9e97ca102e308fd9669ba441510f597bd,19272063,1708470251,0x7c261729906516b7667cf37eb99fb0a9e97ca102e308...,0x7a9bda14323cada0b79bc16ba81f6b59ac85fe98,0x7bea39867e4169dbe237d55c8242a8f2fcdcc387,3.314730,100000.00000,Wrapped Ether,USDC,343923,62365796669,198543,9926177,1135188,2024-02-20 15:04:11


In [9]:
transactions.iloc[0]

blockNumber                                                   20406852
timeStamp                                                   1722191423
hash                 0xe1f15e9e24d6b3bb34ab9ad3c1195d1c88a2aa19b5c1...
nonce                                                            18014
blockHash            0x29f53ae29a668d4f0cea8586fdfec3d37f51599acba4...
from                        0x51c72848c68a965f66fa7a88855f9f7784502a7f
contractAddress             0xa0b86991c6218b36c1d19d4a2e9eb0ce3606eb48
to                          0x7bea39867e4169dbe237d55c8242a8f2fcdcc387
value                                                      20334.06966
tokenName                                                         USDC
tokenSymbol                                                       USDC
tokenDecimal                                                         6
transactionIndex                                                    20
gas                                                             164122
gasPri